In [1]:
from Controller import TrainingController
from Parameters import TrainingParameters
from Utils.SaveUtils import load_parameters
from Utils.PrintUtils import print_big
import tensorflow as tf
import json
import numpy as np
import pandas as pd
from Models import DiCEBinaryDefferentiable
from itertools import chain
from IPython.core.display import display, HTML
from Utils.DiCEHelpers import generate_fake_df, get_trace_with_id, get_longest_trace_row, remove_trail_steps, print_model_prediction_result, remove_tags_for_query_instance

import dice_ml
from dice_ml.utils import helpers

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.4.0-rc0
Eager execution enabled:  True


In [2]:
folder_path = "./SavedModels/%s" % (
    "0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443" #AOW
)

In [3]:
### Initialise controllers
parameters_json = load_parameters(folder_path=folder_path)
parameters = TrainingParameters(**parameters_json)
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)
parameters.load_model_folder_path = folder_path
trainer = TrainingController(parameters = parameters)
trainer.show_model_info()


| Running on /job:localhost/replica:0/task:0/device:CPU:0  

| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012_with_resource/AOW 

| Model loaded successfully from: ./SavedModels/0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443  
Model: "baseline_lstm_with_resource"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  2304      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
lstm_2 (

In [4]:
########### Get example data from trainer ###########
ordered_test_idx = (list(trainer.test_dataset.unbatch().as_numpy_iterator()))
ordered_test_idx.sort()
print_big(len(ordered_test_idx), "Test set length")


====================Test set length====================
| 1309 


In [5]:
########## Get longest delcined trace for testing ##########
declined_df = get_trace_with_id(trainer.dataset.df.iloc[ordered_test_idx], trainer.model.vocab.vocab_to_index('A_DECLINED_COMPLETE'))
longest_declined_trace_row = get_longest_trace_row(declined_df)
longest_declined_trace_row

,trace,trace_vocab,caseid,amount,resource,resource_orig
8464,"[2, 12, 9, 10, 20, 22, 3, 17, 8, 15, 19, 22, 1...","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",200028,5800.0,"[70, 53, 53, 15, 15, 48, 37, 37, 37, 37, 37, 3...","[<SOS>, 112, 112, 10863, 10863, 11169, 11003, ..."


In [6]:
_, example_activities, _, example_resources, example_amount, _ = trainer.dataset.collate_fn([longest_declined_trace_row.index[0]])

In [7]:
 print("========================================Trace========================================")
 print(trainer.model.vocab.list_of_index_to_vocab_2d(example_activities))

========================================Trace========================================
[['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Afhandelen leads_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'A_ACCEPTED_COMPLETE', 'O_SELECTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'O_SELECTED_COMPLETE', 'O_CANCELLED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'O_CANCELLED_COMPLETE', 'O_SELECTED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'O_SENT_BACK_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'W_Valideren aanvraag_COMPLETE', 'W_Nabellen incomplete dossiers_COMPLETE', 'W_Nabellen incomplete dossiers_COMPLETE', 'W_Nabellen incomplete dossiers_COMPLETE', 'W_Nabellen incomplete do

In [8]:
example_idx_activities, example_idx_resources = remove_trail_steps(example_activities, example_resources, 50)
example_vocab_activities = trainer.model.vocab.list_of_index_to_vocab_2d(example_idx_activities)[0]
print("========================================Trace after substraction========================================")
print(example_vocab_activities)

========================================Trace after substraction========================================
['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE']


In [9]:
##### Get model output
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(trainer.model, example_activities_input, example_resources_input, example_amount_input)


========================================Predict result========================================
| Predicted activity with highest probability (0.36) is "W_Afhandelen leads_COMPLETE" 



<PAD>                                      0.000377
<EOS>                                      0.005197
<SOS>                                      0.001689
A_ACCEPTED_COMPLETE                        0.113344
A_ACTIVATED_COMPLETE                       0.001507
A_APPROVED_COMPLETE                        0.000362
A_CANCELLED_COMPLETE                       0.081236
A_DECLINED_COMPLETE                        0.127201
A_FINALIZED_COMPLETE                       0.003128
A_PARTLYSUBMITTED_COMPLETE                 0.004422
A_PREACCEPTED_COMPLETE                     0.050626
A_REGISTERED_COMPLETE                      0.000840
A_SUBMITTED_COMPLETE                       0.017736
O_ACCEPTED_COMPLETE                        0.000727
O_CANCELLED_COMPLETE                       0.000166
O_CREATED_COMPLETE                

In [10]:
predicted_df

,<PAD>,<EOS>,<SOS>,A_ACCEPTED_COMPLETE,A_ACTIVATED_COMPLETE,A_APPROVED_COMPLETE,A_CANCELLED_COMPLETE,A_DECLINED_COMPLETE,A_FINALIZED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,...,O_DECLINED_COMPLETE,O_SELECTED_COMPLETE,O_SENT_BACK_COMPLETE,O_SENT_COMPLETE,W_Afhandelen leads_COMPLETE,W_Beoordelen fraude_COMPLETE,W_Completeren aanvraag_COMPLETE,W_Nabellen incomplete dossiers_COMPLETE,W_Nabellen offertes_COMPLETE,W_Valideren aanvraag_COMPLETE
0,0.000377,0.005197,0.001689,0.113344,0.001507,0.000362,0.081236,0.127201,0.003128,0.004422,...,0.000213,0.028422,0.000866,0.01163,0.362581,0.008908,0.154157,0.007841,0.001002,0.002665


In [11]:
no_need_tags = ['<EOS>', '<SOS>', '<PAD>']
example_idx_activities_no_tag, example_idx_resources_no_tag =  remove_tags_for_query_instance(example_idx_activities, example_idx_resources, trainer.model.vocab.tags_idx(), [trainer.model.resources.index(tag) for tag in no_need_tags])

example_vocab_trace_no_tag = trainer.model.vocab.list_of_index_to_vocab(example_idx_activities_no_tag)
example_vocab_resource_no_tag = [trainer.model.resources[r]  for r in example_idx_resources_no_tag]

example_trace_len_no_tag = len(example_vocab_trace_no_tag)
#### Determine feature names for DiCE ####
activity_feature_names = np.array(["activity_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])
resource_feature_names = np.array(["resource_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])

possible_activities = [ a for a in list(trainer.model.vocab.vocabs) if  not a in no_need_tags]
possible_resources =  [ r for r in list(trainer.model.resources) if  not r in no_need_tags]

possbile_amount = [min(trainer.dataset.df["amount"]), max(trainer.dataset.df["amount"])]



print_big(example_vocab_trace_no_tag ,title="Example activities without tags", num_marks = 30)
print_big(example_vocab_resource_no_tag ,title="Example resources without tags", num_marks = 30)
print_big(example_amount, title="=================Amount=================", num_marks=16)


==============================Example activities without tags==============================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

==============================Example resources without tags==============================
| ['112', '112', '10863'] 

=================================Amount=================================
| [5800.0] 


In [12]:
############ Setting up desired activity ############
desired_activity = 'A_DECLINED_COMPLETE' # A_DECLINED_COMPLETE, A_APPROVED_COMPLETE
print_big(desired_activity, "Desired activity")

dice_binary_model = DiCEBinaryDefferentiable(
    model=trainer.model,
    vocab=trainer.model.vocab,
    resources= trainer.model.resources,
    desired=trainer.model.vocab.vocab_to_index(desired_activity),
    trace_length = len(example_vocab_trace_no_tag),
    sos_idx_activity=trainer.model.vocab.vocab_to_index("<SOS>"),
    sos_idx_resource= trainer.model.resources.index('<SOS>'),
    amount_min = possbile_amount[0],
    amount_max = possbile_amount[1],
    possible_resources=possible_resources,
    possible_activities=possible_activities 
)


====================Desired activity====================
| A_DECLINED_COMPLETE 


In [13]:
fake_df = generate_fake_df(5000, activity_feature_names, resource_feature_names, possible_activities, possible_resources, possbile_amount, example_trace_len_no_tag)
d = dice_ml.Data(dataframe=fake_df, outcome_name="predicted",continuous_features = ['amount'])
m = dice_ml.Model(model=dice_binary_model, backend="TF2")
exp = dice_ml.Dice(d, m)

In [14]:
### Prepare input df
feature_names = activity_feature_names.tolist() + resource_feature_names.tolist() + ['amount']
query_instance = [example_vocab_trace_no_tag + example_vocab_resource_no_tag + example_amount]
example_df = pd.DataFrame(query_instance, columns= feature_names)

In [15]:
example_df

,activity_step_1,activity_step_2,activity_step_3,resource_step_1,resource_step_2,resource_step_3,amount
0,A_SUBMITTED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,A_PREACCEPTED_COMPLETE,112,112,10863,5800.0


In [16]:
## Problem of weight propagation?
dice_exp = exp.generate_counterfactuals(
        example_df,
        total_CFs=1,
        verbose=True,
        min_iter=100,
        max_iter=5000,
        features_to_vary=[
            'amount',
            *activity_feature_names.tolist(),
            *resource_feature_names.tolist(),
        ],
        # desired_class="opposite",
        # yloss_type= "log_loss" # log_loss, hinge_loss, l2_loss
        # algorithm = "DiverseCF", # DiverseCF, RandomInitCF
        # proximity_weight=0.5, # 0.5,
        # diversity_weight=1,# 1.0,
        # init_near_query_instance=True,
        # tie_random = True,
        # categorical_penalty = 1,
        # learning_rate=0.0005,
    )

step 1,  loss=2.92522
step 51,  loss=0.0152328
step 101,  loss=0.0124241
step 151,  loss=0.0112553
step 201,  loss=0.0145335
Diverse Counterfactuals found! total time taken: 00 min 24 sec


In [17]:
dice_exp.visualize_as_dataframe(show_only_changes=True, display_sparse_df=False)

Query instance (original outcome : 0)


,activity_step_1,activity_step_2,activity_step_3,resource_step_1,resource_step_2,resource_step_3,amount,predicted
0,A_SUBMITTED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,A_PREACCEPTED_COMPLETE,112,112,10863,5800.0,0.127



Diverse Counterfactual set without sparsity correction (new outcome:  1.0


,activity_step_1,activity_step_2,activity_step_3,resource_step_1,resource_step_2,resource_step_3,amount,predicted
0,-,-,O_SENT_COMPLETE,11254,-,11029,246.0,0.733


In [18]:
######## Print the example trace ########
print_big(list(example_df.iloc[0]), "Original", num_marks = 50)


==================================================Original==================================================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', '112', '112', '10863', 5800.0] 


In [19]:
######## Print the counterfactual trace ########
if len(dice_exp.final_cfs_df) > 0:
    print_big(list(dice_exp.final_cfs_df.iloc[0][:-1]), "Counterfactual", num_marks=50)
else:
    print_big("Not found!", "Counterfactual")


==================================================Counterfactual==================================================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'O_SENT_COMPLETE', '11254', '112', '11029', 246.0] 


In [20]:
########## Checking if all the permutation are the same (Shouldn't be) ##########
all_cf_are_same = all([all((dice_binary_model.all_cf_input[1][:, 1:] == dice_binary_model.all_cf_input[i][:, 1:]).tolist()[0]) for i in range(len(dice_binary_model.all_cf_input))][1:])
print("All inputs are the same: %s" % (all_cf_are_same))

All inputs are the same: False


In [21]:
########## Checking if all the trace & resource (except amount) are the same (Shouldn't be) ##########
all_trace_and_resource_are_same = all([all((dice_binary_model.all_cf_input[1][:, 1:] == dice_binary_model.all_cf_input[i][:, 1:]).tolist()[0]) for
 i in range(len(dice_binary_model.all_cf_input))][1:])
print("All trace and resource are the same: %s" % (all_trace_and_resource_are_same))

All trace and resource are the same: False


In [42]:
input_cf = exp.cfs[0]

In [45]:
# Continues feature index
exp.encoded_continuous_feature_indexes

[0]

In [51]:
first_cat = exp.encoded_categorical_feature_indexes[0]

<tf.Tensor: shape=(23,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>

In [66]:
maxs = np.argwhere(input_cf[0][first_cat[0]:first_cat[-1]+1] == np.amax(input_cf[0, first_cat[0]:first_cat[-1]+1])).flatten().tolist()

AttributeError: 'list' object has no attribute 'first'

In [50]:
np.argwhere(cf[0, v[0]:v[-1]+1] == np.amax(cf[0, v[0]:v[-1]+1])).flatten().tolist()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]